## Srovnání algoritmů Quicksort a SortX
Naše nulová hypotéza je, že střední hodnota doby běhu Quicksortu je stejná, jako střední hodnota doby běhu algoritmu SortX. Jelikož provádíme párový t-test, tak si spočítáme rozdíly a pro ně pak budeme dělat jednovýběrový t-test s nulovou hypotézou, že střední hodnota těch rozdílů je nulová.

Nejprve načteme data:

In [1]:
import numpy as np
import scipy.stats as ss

qsort = []
xsort = []
with open("times.csv",'r') as file:
    file.readline()
    for line in file:
        qsort.append(int(line.split(sep=',')[1].strip()))
        xsort.append(int(line.split(sep=',')[2].strip()))
print(f"Quicksort data:\t{qsort}")
print(f"SortX data:\t{xsort}")

Quicksort data:	[1014, 1007, 998, 1040, 999, 1030, 980, 1010, 940, 1030, 1000, 990, 1000, 995, 1020, 990, 1040, 1020, 1015, 940]
SortX data:	[980, 995, 960, 1050, 970, 1010, 1005, 1020, 950, 1000, 1025, 970, 965, 980, 1015, 985, 1010, 995, 990, 955]


Spočítáme rozdíly mezi odpovídajícími měřeními, na kterých pak provedeme 1-výběrový t-test.

In [2]:
diffs = [y-x for (y,x) in zip(qsort,xsort)]
print(f"Differences: {diffs}")

Differences: [34, 12, 38, -10, 29, 20, -25, -10, -10, 30, -25, 20, 35, 15, 5, 5, 30, 25, 25, -15]


Pro odhad směrodatné odchylky použijeme výpočet s n-1 stupni volnosti, jelikož už jsme jeden stupeň volnosti ztratili odhadem střední hodnoty.

In [3]:
n = len(diffs)
sample_mean = np.mean(diffs)
sample_stdev = np.std(diffs,ddof=1)

# my_deviations = np.sqrt(np.sum([(x-sample_mean)**2 for x in diffs])/(n-1))

print(f"Sample mean:\t{sample_mean}")
print(f"Sample stdev:\t{sample_stdev}")
print()

Sample mean:	11.4
Sample stdev:	20.661176407124444



### Postup č. 1 (myslím, že tak by to také mělo fungovat)
Řekněme, že budeme používat hladinu významnosti 5%. Tuto hodnotu použijeme k spočtení kritické hodnoty pro t. Výsledek `critical_t_value` nám říká, kolik směrodatných odchylek daleko od střední hodnoty (na libovolnou stranu) musí být nějaká hodnota, aby pravděpodobnost, že ji získáme z té naší distribuce byla menší než 5%.

In [4]:
alpha = 0.05
critical_t_value = ss.t.ppf(1-(alpha/2),n-1)

Nyní nesmím udělat tu chybu jako minule a využít odchylku výběrového průměru (standard error of the mean, SEM), kterou získám tak, že výběrovou směrodatnou odchylku (sample_standard_deviation) vydělím odmocninou z počtu měření.

Rovnou také spočtu tzv. margin of error, který říká (už ve správné škále pro získaná data), jak daleko může být odhadovaná hodnota od té nulové hypotetické tak, aby to ještě bylo v tom konfidenčním intervalu.

In [5]:
sem = ss.sem(diffs)
me = critical_t_value * sem
# my_sem = sample_stdev / np.sqrt(n)

print(f"Standard error of the mean:\t{sem}")
print(f"Margin of error:\t\t{me}")

Standard error of the mean:	4.619979494144513
Margin of error:		9.669728212117384


S tou naměřenou střední hodnotou výběrového průměru by se reálná střední hodnota s 95% spolehlivostí nacházela v následujícím intervalu. Tam ale nespadá hodnota 0, kterou jsme si zadali z nulové hypotézy, takže bychom ji měli zamítnout a SortX je nejspíš opravdu rychlejší. (Mohl by být i pomalejší, ale ta střední hodnota rozdílu je kladná, což napovídá, že se nejspíš jedná o zrychlení)

In [6]:
ci_lower = sample_mean - me
ci_upper = sample_mean + me

print(f"({ci_lower},{ci_upper})")

(1.7302717878826162,21.069728212117383)


### Postup č. 2 (to by měl být standardní postup, jak se to počítá)
Přes t-statistiku, která nám říká, kolik směrodatných odchylek daleko se nacházím od střední hodnoty z nulové hypotézy.

In [7]:
t_score = sample_mean / sem
print(f"T-score: {t_score}")

T-score: 2.467543419716185


Poté spočítám p-hodnotu, která mi říká, jak moc je pravděpodobné, že dostanu výběrový průměr tak daleko od toho z nulové hypotézy, jako jsem ho dostal z těchto vzorků.

In [8]:
p_val1 = 2 * ss.t.cdf(x=-abs(t_score),df=n-1)
p_val2 = 2 * ss.t.sf(abs(t_score),n-1)
print(f"P-value1: {p_val1}")
print(f"P-value2: {p_val2}")

P-value1: 0.02327662950648404
P-value2: 0.02327662950648404


Vyšla mi pravděpodobnost cca 2.3 %, což je méně než hladina 5 %, kterou jsme si stanovili, takže zamítáme hypotézu, že ty algoritmy jsou stejně rychlé as 95% spolehlivostí je SortX výrazně jinak rychlý, než Quicksort. Z dat je vidět, že by se jednalo o zrychlení.